# The Eight Schools Problem: An Intro to Hierarchical Bayesian Analysis

The "eight schools problem" is a very famous example problem that is often used to illustrate hierarchical models and the Bayesian inference process.  This problem was originally described in a 1981 paper by Donald Rubin [2], and it has prominently been features as an example in the third edition of Bayesian Data Analysis [1], to which I will refer below as "BDA3."  The "full Bayesian" model that is commonly used to analyze this problem is simple but very generalizable, and a good understanding of how this model is constructed and analyzed provides a lot of insight into Bayesian data analysis.  Unfortunately, while there are several STAN implementations of this problem available on the internet, none of them provide a lot of context about the problem and simply refer the reader to BDA3.  The **purpose of this document** is to give a reasonably in-depth description of the eight schools problem and the hierarchical model that is commonly used to analyze it.  The document includes:

- An introduction of the eight schools problem and associated data


- A definition of the hierarchical model used to do analysis


- An incomplete analytical derivation of the full model posterior distribution


- A PySTAN implementation of the model



## The Effects of SAT-V Coaching in Eight Schools 

Eight schools hosted special coaching programs for the verbal section of the Scholastic Aptitude Test (SAT-V), and a study was done to investigate the effects of these programs.  For each school, a linear regression was performed where student SAT-V scores were regressed on a binary variable indicating treatment group, with student scores on multiple PSAT sections included as additional controls.  The effects measured by these regresions, along with their sample variances - which are assumed known - are summarized in this table:


| School | Estimated Treatment Effect | Effect Standard Error |
| --- | --- | --- |
| 1 | 28 | 15 |
| 2 | 8 | 10 |
| 3 | -3 | 16 |
| 4 | 7 | 11 |
| 5 | -1 | 9 |
| 6 | 1 | 11 |
| 7 | 18 | 10 |
| 8 | 12 | 18 |


The problem we now face is how to interpret these results and use them to do inference.  One thing we could do is to assume that there is no difference in the effect between schools, and therefore that each school's measured effect is an independent estimate of the one "common" effect $\theta$.  Suppose that we do this and perform a simple Bayesian analysis where our estimates across schools are assumed to be normally distributed and a noninformative (uniform) prior distribution is given to $\theta$.  We will get that our posterior mean for $\theta$ is ~7.7.  Consider a second simple strategy where we assume that each school's effect is different, and we only use data from a given school to estimate the effect at that school.  A simple Bayesian analysis for each school yields school level posterior means that are close to the observed effect sizes, but which are essentially indistinguishable based on the widths of the posterior credible intervals.

These two strategies are nice because of their simplicity, but they are each rather blunt.  The first uses a complete pooling of information across schools, and the second does not allow for any information sharing across schools.  These two strategies also give very different estimates for some of the school level effects.  For example, the first simple strategy estimates the posterior mean of the effect at school 1 as $\approx 7.7$ (the pooled estimate given to all schools), while the second simple strategy estimates it as $\approx 28$.  This result motivates the use of a slightly more complicated model that allows for the "partial" sharing of information across schools...


## Hierarchical Generative Model

Let $\bar{y}_{.j}$ denote the estimated treatment effect for school $j$, where $j=1, ..., 8$.  Let $\sigma_j^2$ denote the sampling variance associated with the estimate at school $j$.  Note that this notation is used to maintain consistency with the presentation of this model in BDA3, and the "dot" notation used to denote the within group sample mean is classically used in analysis of variance.  We now assume the following hierarchical model describing the generative process that creates these $\theta_j$ values:

$$
\overline{y_{.j}} \sim N(\theta_j, \sigma_j^2)
$$

Where:

$$
\overline{y_{.j}} = \frac{1}{n_j} \sum_{i=1}^{n_j} y_{ij}
$$
$$
\sigma_j^2 = \frac{\sigma^2}{n_j}
$$

Here comes the hierarchical part - we assume that the $\theta_j$ values are themselves sampled independently from a Normal distribution with shared hyperparameters $\mu$ and $\tau$:

$$
\theta_j \sim_{iid} N(\mu, \tau^2)
$$


## Posterior Distribution over Model Parameters

Using a Bayesian approach with the model specified above, the full posterior distribution over all parameters can be factorized using the probability chain rule:

$p(\theta, \mu, \tau | y) = p(\theta|\mu, \tau, y)p(\mu|\tau, y)p(\tau|y)$


Specifying $p(\mu, \tau) = p(\mu | \tau)p(\tau)$ - the prior distribution over our hyperparameters - will give us enough information to analytically solve for the posterior.  We will do this below as we derive the second and third terms of the factorization.  We don't need any information from the prior in order to derive the first term $p(\theta|\mu, \tau, y)$, since we are already conditioning on both $\mu$ and $\tau$.  Our model assumptions give us a prior for each $\theta_j$, and a sampling distribution of $y_{ij}$ given $\theta_j$ (a likelihood).  Using these to solve for the posterior on $\theta$ yields:

$$
\theta|\mu, \tau, y \sim N(\hat{\theta}_j, V_j)
$$

Where:

$$
\hat{\theta}_j = \frac{\frac{1}{\sigma_j^2} \bar{y}_{.j} + \frac{1}{\tau^2} \mu  }{\frac{1}{\sigma_j^2} + \frac{1}{\tau^2}}
$$

$$
V_j = \frac{1}{ \frac{1}{\sigma_j^2} + \frac{1}{\tau^2}  }
$$

Now, we **assume a uniform conditional prior** on $\mu|\tau$, or in other words assume that $p(\mu | \tau) = c$.  This allows us to factor the joint prior over these hyperparameters as $p(\mu, \tau) = p(\mu|\tau)p(\tau) \propto p(\tau)$.  The assignment of a "noninformative" prior over $\mu|\tau$ is a **modeling decision** that is motivated by the fact that the data we observe will provide us with a lot of information about $\mu$ (pg. 115).  This decision implies the following conditional distribution for $\mu|\tau, y$, the second term in the factorization of the posterior:

$$
\mu|\tau, y \sim N(\hat{\mu}, V_\mu)
$$

Where:

$$
\hat{\mu} = \frac{  \sum_{j=1}^J \frac{1}{\sigma^2_j + \tau^2} \bar{y}_{.j}    }{  \sum_{j=1}^J \frac{1}{\sigma^2_j + \tau^2}  }
$$

$$
V_\mu^{-1} = \sum_{j=1}^J \frac{1}{\sigma_j^2 + \tau^2}
$$


Finally, we deterine the third term of the factorization, the conditional posterior of $\tau|y$.  We can express this as:

$$
p(\tau|y) = \frac{p(\mu, \tau|y)}{p(\mu|\tau, y)}
$$



$$
\propto \frac{p(\tau) \prod_{j=1}^J N(\bar{y}_{.j}|\hat{\mu}, \sigma_j^2 + \tau^2) }{N(\hat{\mu}|\hat{\mu}, V_u)}
$$
$$
\propto p(\tau) V_u^{1/2} \prod_{j=1}^J (\sigma_j^2 + \tau^2)^{-1/2} exp\left(  - \frac{(\bar{y}_{.j} - \hat{\mu})^2} {2(\sigma_j^2 + \tau^2)} \right)
$$

We **assume a uniform prior** on $\tau$: $p(\tau) \propto c$ for $\tau > 0$.  This implies:

$$
p(\tau|y) \propto V_u^{1/2} \prod_{j=1}^J (\sigma_j^2 + \tau^2)^{-1/2} exp\left(  - \frac{(\bar{y}_{.j} - \hat{\mu})^2} {2(\sigma_j^2 + \tau^2)} \right)
$$

We now have an analytical expression (up to constant scaling) of our posterior $p(\theta, \mu, \tau | y)$.  We see that the posterior is quite a complicated function, and instead of attempting to integrate it (e.g. to normalize it) we will prefer to analyze it numerically.  This can be done in a straightforward way - we first sample $\tau$, which can be done by computing $p(\tau|y)$ for a uniformly spaced grid of $\tau$ values and using the inverse CDF method (see [here](https://stephens999.github.io/fiveMinuteStats/inverse_transform_sampling.html)) on this discretized distribution of $\tau$.  We then sample $\mu$ and then $\theta$ from their associated Normal distributions.

--------------------------------------------

To summarize, we:

- expressed the posterior distribution for the parameters of our hierarchical model as a factorization of three conditional posteriors

- Assumed noninformative uniform priors for $p(\mu|\tau)$ and $p(\tau)$

- Used the the assumed priors to analytically express the posterior factors


The full details of the posterior derivation are omitted here for the sake of brevity, but are included in chapter 5 ofi BDA3.

## Tau and Pooled vs. Separate School Estimates  

The procedure described above allows us to do inference on the $\tau$ parameter of the model after specifying a prior distribution over it.  However, if we were to set $\tau = 0$ we would find that the distributions for the $\theta_j$ values given by our Bayesian procedure would all be centered around the pooled estimate of $\theta_j = 7.7$ that is given by the the first "simple strategy" described above.  On the other hand, if we take $\tau \rightarrow \infty$, we find that the distribution estimates over $\theta_j$ parameters are centered about the separate estimates given by the second "simple strategy."  We can thus think of $\tau$ as a sort of knob that we could turn to give us a different interpolation between the fully pooled and fully separated strategies for estimating the $\theta_j$ values.  This result makes sense given the interpretation of $\tau$ in the model - small values of $\tau$ imply that the different school effects are (in all probability) very close to each other.


This is mentioned only as a curiosity, but it illustrates the flexibility and power of the hierarchical model we've constructed.




## Automatically Sampling from the Model Posterior with STAN

In BDA3, Gelman et. all mention the sampling method described above as a way to numerically analyze the posterior distribution of the model ([1], pg. 118).  However, in general we can avoid the posterior derivations necessary for this by using a dedicated Bayesian software package like STAN.  STAN allows us to build a model by declaring a likelihood and prior, and then automatically sample from the posterior after supplying data to the model.  STAN is very powerful in that it allows us to easily experiment with different model specifications without having to spend time coming up with a good way to sample from the posterior - it all happens "automagically."  The magic is supplied by way of a powerful Markov Chain Monte Carlo algorithm known as Hamiltonian Monte Carlo, which is how STAN draws samples.  STAN has APIs for R and Python (among other languages), and PyStan is used below to analyze the hierarchical model described above.

In the code below, the priors on $\mu$ and $\tau$ are not uniform, as they are assumed in the general model presented in BDA3.  Rather, these are the priors suggested in the original Rubin paper [2]. Unfortunately, these priors give the posterior a geometry that is problematic for the STAN sampler, which is evidenced by the divergence warnings we get when we sample.  This illustrates the difficulty of modeling with a program as flexible as STAN - we can end up building models that yield posteriors that are very difficult to sample from.  [Here](https://mc-stan.org/users/documentation/case-studies/divergences_and_bias.html) is a phenomenal blog post that goes into depth about how to diagnose and address the issues that arise when using STAN to do inference.



Here are some additional helpful resources related to STAN and PyStan:


- The Stan User Manual is [here](https://mc-stan.org/docs/2_29/stan-users-guide-2_29.pdf).


- A wonderful paper that gives some intuition for Hamiltonian Monte Carlo sampling is [here](https://arxiv.org/pdf/1701.02434.pdf?ref=https://githubhelp.com)


- The PyStan GitHub repository (which includes some documentation) can be [here](https://github.com/stan-dev/pystan).


In [29]:
import pystan
import pandas as pd


# model specification - this can be read directly off a file by PyStan (see docs)
model_code = """
data {
  int<lower=0> J;
  real y[J];
  real<lower=0> sigma[J];
}

parameters {
  real mu;
  real theta[J];
  real<lower=0> tau;
}

model {
  mu ~ normal(0, 5);
  tau ~ cauchy(0, 5);
  theta ~ normal(mu, tau);
  y ~ normal(theta, sigma);
}
"""

# data to suppply to the sampler
model_data = dict(
    J=8,
    y=[28,  8, -3,  7, -1,  1, 18, 12], 
    sigma=[15, 10, 16, 11,  9, 11, 10, 18],
)

# compile model
sm = pystan.StanModel(model_code=model_code)

# Train the model and generate samples
fit = sm.sampling(
    data=model_data,
    iter=1000,
    chains=4,
    warmup=500,
    thin=1,
    control=dict(adapt_delta=0.8),
    seed=101
)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3955c9c7b33c6da5b960bfeb1d23fde0 NOW.


In [30]:
# read into Pandas
summary_dict = fit.summary()

res_df = pd.DataFrame(
    summary_dict['summary'], 
    columns=summary_dict['summary_colnames'], 
    index=summary_dict['summary_rownames']
)

# print some quantiles from our posterior 
res_df


,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
mu,8.011503,0.313131,4.451974,-0.648883,5.039417,7.950923,11.005299,16.774943,202.140699,1.022306
theta[1],9.667488,0.326659,6.253741,-0.861679,5.649364,9.105090,12.805034,24.874256,366.513844,1.006775
theta[2],8.094177,0.271956,5.399693,-2.054649,4.485995,7.963993,11.433316,19.309956,394.220558,1.005319
theta[3],7.342270,0.325473,6.118242,-5.701914,3.626793,7.478395,11.485344,18.995560,353.364791,1.011964
theta[4],7.892158,0.268866,5.726444,-3.530565,4.389887,7.750657,11.341168,19.213511,453.627152,1.013458
theta[5],6.337915,0.360894,5.507145,-5.235362,2.987551,6.629630,10.101849,16.266123,232.859036,1.019186
theta[6],7.091334,0.336453,5.573763,-5.034182,3.692490,7.150886,10.881964,17.533419,274.441023,1.014592
theta[7],9.420656,0.304090,5.554803,-0.630173,5.730028,9.078182,12.626336,21.262046,333.683169,1.008134
theta[8],8.500329,0.306907,6.482098,-4.160689,4.533094,8.334522,12.109275,22.559455,446.086254,1.007343
tau,4.097441,0.331508,3.283590,0.519690,1.705793,3.222751,5.540268,12.619995,98.109513,1.031000


## References

[1] Andrew Gelman, John B. Carlin, Hal S. Stern, David B. Dunson, Aki Vehtari, and Donald B.
Rubin. *Bayesian Data Analysis*. Chapman and Hall/CRC, New York, NY, 3
edition, 2013.

[2] Rubin, Donald B. “Estimation in Parallel Randomized Experiments.” Journal of Educational Statistics, vol. 6, no. 4, [Sage Publications, Inc., American Educational Research Association, American Statistical Association], 1981, pp. 377–401, https://doi.org/10.2307/1164617.